# Assignment 3

In [2]:
# Imports
import pandas as pd
import os
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience

### Connect to the Neo4j server and to use Neo4j GDS

In [12]:
NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")

In [16]:
NEO4J_AUTH = ( "neo4j",  "12072003") ### Make sure the password is yours

In [14]:
driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)

In [15]:
gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH)

In [ ]:
### Cypher Query to return clientId and clusterId
CALL gds.wcc.stream('clientClusters')
YIELD nodeId, componentId
RETURN gds.util.asNode(nodeId).id AS clientId, componentId AS clusterId